# LIMPIEZA DE LA BASE DE DATOS ENAHO PANEL 2015-2019
### Josué Daniel Caldas Velásquez

### Objetivo

. El presente trabajo centra su atención en saber si los jóvenes recién egresados de secundaria lograrán acceder a educación superior hasta un año después de su egreso. Entonces, la variable acceso a educación superior se operacionaliza como variable dummy donde 1 indica que sí logró acceder a educación superior dentro del año posterior a su egreso de educación básica secundaria, y 0 indica que no logró acceder. 

Para lograr este objetivo se usa la base Enaho Panel 2015-2019, módulo educación. Esta base es dividida en cuatro sub bases para periodos de dos años: 2015-2016, 2016-2017, 2017-2018, y 2018_2019. El objetivo es saber si un joven recién egresado de secundaria en el año x accedió a educación superior hasta el año x+1. Por ejemplo, saber si un joven egresado de secundaria en el año 2015, accedió a educación superior hasta el año 2016. De este modo, se crea una variable dummy con el acceso o no. Para identificar a las observaciones, y asegurar la comparabilidad, se genera un 'id_personal' y un 'id_vivienda'. Así, el 'id_personal' permite identificar a las observaciones en los periodos de dos años, para saber si accedieron a educación superior o no. 

Cabe mencionar una aclaración:

Por acceso a educación superior se entiende el acceso a educación superior universitaria o a educación superior no universitaria (técnica).

In [1]:
# Importamos librerias

import pandas as pd
import numpy as np
import re
import pyarrow.parquet as pq
import pyarrow as pa
import wbgapi as wb
import nbconvert

## PARTE 1. TRANSFORMACIÓN DE DATOS

#### Importamos nuestros datos

In [2]:
educacion = pd.read_csv("educacion_enaho.csv", encoding = "latin-1")

C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/2583387138.py:1: DtypeWarning: Columns (16,17,21,22,25,28,30,31,32,33,34,35,38,39,40,41,42,45,46,47,48,49,50,51,52,54,55,59,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,85,88,89,90,91,93,94,96,99,103,104,105,107,109,110,111,112,113,114,115,117,118,119,120,121,122,123,124,125,126,127,128,129,131,141,142,143,144,147,167,168,170,171,172,173,177,178,181,182,189,190,191,192,193,194,195,196,198,199,200,201,202,203,204,207,208,209,210,211,212,213,214,215,216,217,222,223,224,225,226,227,229,230,231,274,283,284,288,289,292,293,295,298,299,300,301,302,303,306,307,308,309,310,313,314,315,316,317,318,319,320,321,325,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,352,353,355,356,358,360,361,362,363,364,366,368,370,372,374,376,377,378,379,380,381,382,384,385,386,387,388,389,390,391,392,393,394,395,396,398,408,409,410,411,414,488,489,490,491,492,493,494,503,504,506,507,509,525,526,

In [3]:
educacion.shape

(473666, 2214)

### Gestionamos la base de datos 'educacion'

#### Limpieza

Nos quedamos solo con las variables que nos permiten identificar a jóvenes recién egresados de educación secundaria:

'hogar_1x' = código de hogar
'codperso_1x' = código de la persona
'conglome_1x' = conglomerado al que pertenece el hogar
'vivienda_1x' = identificador de vivienda
'p301a_1x' = último nivel de estudios que aprobó
'p304a_1x' = nivel de estudios al que asistió el año pasado

In [4]:
#Nos quedamos con las variables de importancia

lista_edu = ['hogar_15', 'hogar_16', 'hogar_17', 'hogar_18', 'hogar_19', 
             'codperso_15', 'codperso_16', 'codperso_17', 'codperso_18', 'codperso_19',
             'conglome_15', 'conglome_16', 'conglome_17', 'conglome_18', 'conglome_19',
             'vivienda_15', 'vivienda_16', 'vivienda_17', 'vivienda_18', 'vivienda_19', 
             'p301a_15', 'p301a_16', 'p301a_17', 'p301a_18', 'p301a_19',
             'p304a_15', 'p304a_16', 'p304a_17', 'p304a_18', 'p304a_19']
edu = educacion[lista_edu].copy().reset_index()
edu.columns= edu.columns.str.lower()
edu

,index,hogar_15,hogar_16,hogar_17,hogar_18,hogar_19,codperso_15,codperso_16,codperso_17,codperso_18,...,p301a_15,p301a_16,p301a_17,p301a_18,p301a_19,p304a_15,p304a_16,p304a_17,p304a_18,p304a_19
0,0,11.0,11.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,primaria incompleta,primaria incompleta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,11.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,sin nivel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,11.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,secundaria completa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,11.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,secundaria completa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,11.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,secundaria completa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473661,473661,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sin nivel,NaN,NaN,NaN,NaN,NaN
473662,473662,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sup. no univ. incompleta,NaN,NaN,NaN,NaN,superior no universitaria
473663,473663,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,primaria completa,NaN,NaN,NaN,NaN,NaN
473664,473664,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,sup. no univ. completa,NaN,NaN,NaN,NaN,NaN


Llegados a este punto, debemos generar los siguientes valores:

// Para el año 2015: seleccionar observaciones que tengan secundaria completa y sean recien egresados de ese nivel. Nombre: 'edu_2015_comp'

_______________________________________________________________________________________________________________________________

// Para el año 2016: dos sub grupos.

Primer subgrupo: Que tenga cualquier nivel de estudios, para comparar con los datos del 2015, y ver si los recién egresados de secundaria lograron acceder a educación superior. Nombre: 'edu_2016'

Segundo subgrupo: Seleccionar observaciones que tengan secundaria completa y sean recien egresados de ese nivel. Nombre: 'edu_2016_comp'

_______________________________________________________________________________________________________________________________

// Para el año 2017: dos sub grupos.

Primer subgrupo: Que tenga cualquier nivel de estudios, para comparar con los datos del segundo subgrupo de 2016, y ver si los recién egresados de secundaria lograron acceder a educación superior. Nombre: 'edu_2017'

Segundo subgrupo: Seleccionar observaciones que tengan secundaria completa y sean recien egresados de ese nivel. Nombre: 'edu_2017_comp'

______________________________________________________________________________________________________________________________

// Para el año 2018: dos sub grupos

Primer subgrupo: Que tenga cualquier nivel de estudios, para comparar con los datos del segundo subgrupo de 2017, y ver si los recién egresados de secundaria lograron acceder a educación superior. Nombre: 'edu_2018'

Segundo subgrupo: Seleccionar observaciones que tengan secundaria completa y sean recien egresados de ese nivel. Nombre: 'edu_2018_comp'
______________________________________________________________________________________________________________________________

// Para el año 2019: seleccionar observaciones que tengan cualquier nivel de estudios, para comparar con los datos del segundo subgrupo del 2018, y ver si los recién egresados de secudnaria lograron acceder a educación superior. Nombre: 'edu_2019'

In [38]:
# Generamos las listas con las variables para las datasets de cada año
# Primero creamos una sub base general por año, y luego, cuando limpiamos cada año, generamos las sub bases adicionales para los años 2016 y 2017

lista_2015 = ['hogar_15',
              'codperso_15',
              'conglome_15', 
              'vivienda_15',
              'p301a_15',
             'p304a_15']

lista_2016 = ['hogar_16',
              'codperso_16',
              'conglome_16', 
              'vivienda_16',
              'p301a_16',
             'p304a_16']

lista_2017 = ['hogar_17',
              'codperso_17',
              'conglome_17', 
              'vivienda_17',
              'p301a_17',
             'p304a_17']

lista_2018 = ['hogar_18',
              'codperso_18',
              'conglome_18', 
              'vivienda_18',
              'p301a_18',
             'p304a_18']

lista_2019 = ['hogar_19',
              'codperso_19',
              'conglome_19', 
              'vivienda_19',
              'p301a_19',
             'p304a_19']

In [39]:
# Creamos las bases para cada año

edu_2015 = edu[lista_2015].copy().reset_index()
edu_2016 = edu[lista_2016].copy().reset_index()
edu_2017 = edu[lista_2017].copy().reset_index()
edu_2018 = edu[lista_2018].copy().reset_index()
edu_2019 = edu[lista_2019].copy().reset_index()

#### Trabajamos con la sub base 2015

In [40]:
# Seleccionamos observaciones cuyo último nivel de estudios aprobado fue secundaria
secundaria = ['secundaria completa']
edu_2015_comp = edu_2015[edu.p301a_15.isin(secundaria) == True]

# Seleccionamos observaciones que el año anterior a la encuesta asistieron a secundaria
secundaria1 = ['secundaria']
edu_2015_comp = edu_2015_comp[edu.p304a_15.isin(secundaria1) == True]
edu_2015_comp.shape

C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/2820792585.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2015_comp = edu_2015_comp[edu.p304a_15.isin(secundaria1) == True]


(1665, 7)

In [41]:
#Borramos las filas con valores perdidos

edu_2015_comp = edu_2015_comp.dropna()
edu_2015_comp

,index,hogar_15,codperso_15,conglome_15,vivienda_15,p301a_15,p304a_15
210,210,11.0,4.0,8232.0,6.0,secundaria completa,secundaria
417,417,11.0,4.0,7690.0,25.0,secundaria completa,secundaria
662,662,11.0,3.0,7259.0,29.0,secundaria completa,secundaria
776,776,11.0,5.0,5799.0,33.0,secundaria completa,secundaria
783,783,11.0,3.0,9000.0,121.0,secundaria completa,secundaria
...,...,...,...,...,...,...,...
454171,454171,11.0,4.0,9671.0,63.0,secundaria completa,secundaria
457377,457377,11.0,3.0,9855.0,132.0,secundaria completa,secundaria
463268,463268,11.0,2.0,10126.0,102.0,secundaria completa,secundaria
463447,463447,11.0,3.0,10135.0,40.0,secundaria completa,secundaria


In [42]:
# Quitamos el .0 de hogar, codperso, conglome y vivienda

edu_2015_comp['codperso_15'] = edu_2015_comp['codperso_15'].astype(str).replace('\.0', '', regex=True)
edu_2015_comp['vivienda_15'] = edu_2015_comp['vivienda_15'].astype(str).replace('\.0', '', regex=True)
edu_2015_comp['conglome_15'] = edu_2015_comp['conglome_15'].astype(str).replace('\.0', '', regex=True)
edu_2015_comp['hogar_15'] = edu_2015_comp['hogar_15'].astype(str).replace('\.0', '', regex=True)
edu_2015_comp

,index,hogar_15,codperso_15,conglome_15,vivienda_15,p301a_15,p304a_15
210,210,11,4,8232,6,secundaria completa,secundaria
417,417,11,4,7690,25,secundaria completa,secundaria
662,662,11,3,7259,29,secundaria completa,secundaria
776,776,11,5,5799,33,secundaria completa,secundaria
783,783,11,3,9000,121,secundaria completa,secundaria
...,...,...,...,...,...,...,...
454171,454171,11,4,9671,63,secundaria completa,secundaria
457377,457377,11,3,9855,132,secundaria completa,secundaria
463268,463268,11,2,10126,102,secundaria completa,secundaria
463447,463447,11,3,10135,40,secundaria completa,secundaria


In [43]:
# Ahora generamos el código de identificacion personal: conglomerado + vivienda + hogar + codigo persona

edu_2015_comp['personal_id'] = edu_2015_comp['conglome_15'].astype(str) + edu_2015_comp['vivienda_15'].astype(str) + edu_2015_comp['hogar_15'].astype(str) + edu_2015_comp['codperso_15'].astype(str)

In [44]:
# Generamos el código de identificacion de vivienda: conglomerado + vivienda + hogar

edu_2015_comp['vivienda_id'] = edu_2015_comp['conglome_15'].astype(str) + edu_2015_comp['vivienda_15'].astype(str) + edu_2015_comp['hogar_15'].astype(str)
edu_2015_comp

,index,hogar_15,codperso_15,conglome_15,vivienda_15,p301a_15,p304a_15,personal_id,vivienda_id
210,210,11,4,8232,6,secundaria completa,secundaria,82326114,8232611
417,417,11,4,7690,25,secundaria completa,secundaria,769025114,76902511
662,662,11,3,7259,29,secundaria completa,secundaria,725929113,72592911
776,776,11,5,5799,33,secundaria completa,secundaria,579933115,57993311
783,783,11,3,9000,121,secundaria completa,secundaria,9000121113,900012111
...,...,...,...,...,...,...,...,...,...
454171,454171,11,4,9671,63,secundaria completa,secundaria,967163114,96716311
457377,457377,11,3,9855,132,secundaria completa,secundaria,9855132113,985513211
463268,463268,11,2,10126,102,secundaria completa,secundaria,10126102112,1012610211
463447,463447,11,3,10135,40,secundaria completa,secundaria,1013540113,101354011


#### Trabajamos con la sub base 2016

In [45]:
#Borramos las filas con valores perdidos

edu_2016 = edu_2016.dropna()
edu_2016

,index,hogar_16,codperso_16,conglome_16,vivienda_16,p301a_16,p304a_16
5,5,11.0,2.0,7007.0,138.0,secundaria incompleta,secundaria
26,26,11.0,5.0,9249.0,31.0,primaria incompleta,primaria
62,62,11.0,3.0,9034.0,48.0,sup. no univ. incompleta,superior no universitaria
72,72,11.0,4.0,1150.0,100.0,secundaria incompleta,secundaria
104,104,11.0,3.0,2282.0,80.0,secundaria incompleta,secundaria
...,...,...,...,...,...,...,...
469134,469134,11.0,5.0,10381.0,66.0,educación inicial,educación inicial
469135,469135,11.0,4.0,10381.0,56.0,secundaria completa,secundaria
469137,469137,11.0,3.0,10381.0,66.0,secundaria incompleta,secundaria
469143,469143,11.0,7.0,10381.0,56.0,educación inicial,educación inicial


In [46]:
# Quitamos el .0 de hogar, codperso, conglome y vivienda

edu_2016['codperso_16'] = edu_2016['codperso_16'].astype(str).replace('\.0', '', regex=True)
edu_2016['vivienda_16'] = edu_2016['vivienda_16'].astype(str).replace('\.0', '', regex=True)
edu_2016['conglome_16'] = edu_2016['conglome_16'].astype(str).replace('\.0', '', regex=True)
edu_2016['hogar_16'] = edu_2016['hogar_16'].astype(str).replace('\.0', '', regex=True)
edu_2016

,index,hogar_16,codperso_16,conglome_16,vivienda_16,p301a_16,p304a_16
5,5,11,2,7007,138,secundaria incompleta,secundaria
26,26,11,5,9249,31,primaria incompleta,primaria
62,62,11,3,9034,48,sup. no univ. incompleta,superior no universitaria
72,72,11,4,1150,100,secundaria incompleta,secundaria
104,104,11,3,2282,80,secundaria incompleta,secundaria
...,...,...,...,...,...,...,...
469134,469134,11,5,10381,66,educación inicial,educación inicial
469135,469135,11,4,10381,56,secundaria completa,secundaria
469137,469137,11,3,10381,66,secundaria incompleta,secundaria
469143,469143,11,7,10381,56,educación inicial,educación inicial


In [47]:
# Ahora generamos el código el 'id' de cada observación: conglomerado + vivienda + hogar + codigo persona

edu_2016['personal_id'] = edu_2016['conglome_16'].astype(str) + edu_2016['vivienda_16'].astype(str) + edu_2016['hogar_16'].astype(str) + edu_2016['codperso_16'].astype(str)

In [48]:
# Generamos también la variable 'id_vivienda': conglomerado + vivienda + hogar
edu_2016['vivienda_id'] = edu_2016['conglome_16'].astype(str) + edu_2016['vivienda_16'].astype(str) + edu_2016['hogar_16'].astype(str)
edu_2016

,index,hogar_16,codperso_16,conglome_16,vivienda_16,p301a_16,p304a_16,personal_id,vivienda_id
5,5,11,2,7007,138,secundaria incompleta,secundaria,7007138112,700713811
26,26,11,5,9249,31,primaria incompleta,primaria,924931115,92493111
62,62,11,3,9034,48,sup. no univ. incompleta,superior no universitaria,903448113,90344811
72,72,11,4,1150,100,secundaria incompleta,secundaria,1150100114,115010011
104,104,11,3,2282,80,secundaria incompleta,secundaria,228280113,22828011
...,...,...,...,...,...,...,...,...,...
469134,469134,11,5,10381,66,educación inicial,educación inicial,1038166115,103816611
469135,469135,11,4,10381,56,secundaria completa,secundaria,1038156114,103815611
469137,469137,11,3,10381,66,secundaria incompleta,secundaria,1038166113,103816611
469143,469143,11,7,10381,56,educación inicial,educación inicial,1038156117,103815611


In [49]:
# Ahora generamos el segundo sub grupo para el año 2016: personas que tienen secundaria completa y son recien egresadas de ese nivel

# Seleccionamos observaciones cuyo último nivel de estudios aprobado fue secundaria
secundaria = ['secundaria completa']
edu_2016_comp = edu_2016[edu.p301a_16.isin(secundaria) == True]

# Seleccionamos observaciones que el año anterior a la encuesta asistieron a secundaria
secundaria161 = ['secundaria']
edu_2016_comp = edu_2016_comp[edu.p304a_16.isin(secundaria1) == True]
edu_2016_comp.shape

C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/2439247114.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2016_comp = edu_2016[edu.p301a_16.isin(secundaria) == True]
C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/2439247114.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2016_comp = edu_2016_comp[edu.p304a_16.isin(secundaria1) == True]


(1860, 9)

#### Trabajamos con la base 2017

In [50]:
#Borramos las filas con valores perdidos

edu_2017 = edu_2017.dropna()
edu_2017

,index,hogar_17,codperso_17,conglome_17,vivienda_17,p301a_17,p304a_17
5,5,11.0,2.0,7007.0,138.0,secundaria incompleta,secundaria
10,10,11.0,3.0,9810.0,32.0,primaria incompleta,primaria
19,19,11.0,3.0,8676.0,149.0,secundaria incompleta,secundaria
20,20,11.0,3.0,6961.0,93.0,educación inicial,educación inicial
23,23,11.0,4.0,5310.0,98.0,primaria incompleta,primaria
...,...,...,...,...,...,...,...
472145,472145,11.0,7.0,10757.0,117.0,primaria incompleta,primaria
472147,472147,11.0,4.0,10757.0,117.0,sup. univ. incompleta,superior universitaria
472200,472200,11.0,1.0,10769.0,115.0,sup. no univ. incompleta,superior no universitaria
472202,472202,11.0,5.0,10769.0,12.0,secundaria completa,secundaria


In [51]:
# Quitamos el .0 de hogar, codperso, conglome y vivienda

edu_2017['codperso_17'] = edu_2017['codperso_17'].astype(str).replace('\.0', '', regex=True)
edu_2017['vivienda_17'] = edu_2017['vivienda_17'].astype(str).replace('\.0', '', regex=True)
edu_2017['conglome_17'] = edu_2017['conglome_17'].astype(str).replace('\.0', '', regex=True)
edu_2017['hogar_17'] = edu_2017['hogar_17'].astype(str).replace('\.0', '', regex=True)
edu_2017

,index,hogar_17,codperso_17,conglome_17,vivienda_17,p301a_17,p304a_17
5,5,11,2,7007,138,secundaria incompleta,secundaria
10,10,11,3,9810,32,primaria incompleta,primaria
19,19,11,3,8676,149,secundaria incompleta,secundaria
20,20,11,3,6961,93,educación inicial,educación inicial
23,23,11,4,5310,98,primaria incompleta,primaria
...,...,...,...,...,...,...,...
472145,472145,11,7,10757,117,primaria incompleta,primaria
472147,472147,11,4,10757,117,sup. univ. incompleta,superior universitaria
472200,472200,11,1,10769,115,sup. no univ. incompleta,superior no universitaria
472202,472202,11,5,10769,12,secundaria completa,secundaria


In [52]:
# Ahora generamos el código el 'id' de cada observación: conglomerado + vivienda + hogar + codigo persona

edu_2017['personal_id'] = edu_2017['conglome_17'].astype(str) + edu_2017['vivienda_17'].astype(str) + edu_2017['hogar_17'].astype(str) + edu_2017['codperso_17'].astype(str)

# Generamos también la variable 'id_vivienda': conglomerado + vivienda + hogar
edu_2017['vivienda_id'] = edu_2017['conglome_17'].astype(str) + edu_2017['vivienda_17'].astype(str) + edu_2017['hogar_17'].astype(str)
edu_2017

,index,hogar_17,codperso_17,conglome_17,vivienda_17,p301a_17,p304a_17,personal_id,vivienda_id
5,5,11,2,7007,138,secundaria incompleta,secundaria,7007138112,700713811
10,10,11,3,9810,32,primaria incompleta,primaria,981032113,98103211
19,19,11,3,8676,149,secundaria incompleta,secundaria,8676149113,867614911
20,20,11,3,6961,93,educación inicial,educación inicial,696193113,69619311
23,23,11,4,5310,98,primaria incompleta,primaria,531098114,53109811
...,...,...,...,...,...,...,...,...,...
472145,472145,11,7,10757,117,primaria incompleta,primaria,10757117117,1075711711
472147,472147,11,4,10757,117,sup. univ. incompleta,superior universitaria,10757117114,1075711711
472200,472200,11,1,10769,115,sup. no univ. incompleta,superior no universitaria,10769115111,1076911511
472202,472202,11,5,10769,12,secundaria completa,secundaria,1076912115,107691211


In [53]:
# Ahora generamos el segundo sub grupo para el año 2017: personas que tienen secundaria completa y son recien egresadas de ese nivel

# Seleccionamos observaciones cuyo último nivel de estudios aprobado fue secundaria
secundaria = ['secundaria completa']
edu_2017_comp = edu_2017[edu.p301a_17.isin(secundaria) == True]

# Seleccionamos observaciones que el año anterior a la encuesta asistieron a secundaria
secundaria1 = ['secundaria']
edu_2017_comp = edu_2017_comp[edu.p304a_17.isin(secundaria1) == True]
edu_2017_comp.shape

C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/3933631426.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2017_comp = edu_2017[edu.p301a_17.isin(secundaria) == True]
C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/3933631426.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2017_comp = edu_2017_comp[edu.p304a_17.isin(secundaria1) == True]


(1768, 9)

#### Trabajamos con la base 2018

In [54]:
#Borramos las filas con valores perdidos

edu_2018 = edu_2018.dropna()
edu_2018

,index,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18
16,16,11.0,3.0,5636.0,20.0,primaria incompleta,primaria
28,28,11.0,3.0,7178.0,139.0,sup. no univ. completa,superior no universitaria
35,35,11.0,3.0,10322.0,109.0,primaria completa,primaria
38,38,11.0,4.0,8690.0,6.0,primaria incompleta,primaria
56,56,11.0,5.0,6571.0,194.0,primaria incompleta,primaria
...,...,...,...,...,...,...,...
472141,472141,11.0,3.0,10757.0,117.0,sup. univ. completa,superior universitaria
472145,472145,11.0,7.0,10757.0,117.0,primaria incompleta,primaria
472147,472147,11.0,4.0,10757.0,117.0,sup. univ. incompleta,superior universitaria
472200,472200,11.0,1.0,10769.0,115.0,sup. univ. incompleta,superior universitaria


In [55]:
# Quitamos el .0 de hogar, codperso, conglome y vivienda

edu_2018['codperso_18'] = edu_2018['codperso_18'].astype(str).replace('\.0', '', regex=True)
edu_2018['vivienda_18'] = edu_2018['vivienda_18'].astype(str).replace('\.0', '', regex=True)
edu_2018['conglome_18'] = edu_2018['conglome_18'].astype(str).replace('\.0', '', regex=True)
edu_2018['hogar_18'] = edu_2018['hogar_18'].astype(str).replace('\.0', '', regex=True)
edu_2018

,index,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18
16,16,11,3,5636,20,primaria incompleta,primaria
28,28,11,3,7178,139,sup. no univ. completa,superior no universitaria
35,35,11,3,10322,109,primaria completa,primaria
38,38,11,4,8690,6,primaria incompleta,primaria
56,56,11,5,6571,194,primaria incompleta,primaria
...,...,...,...,...,...,...,...
472141,472141,11,3,10757,117,sup. univ. completa,superior universitaria
472145,472145,11,7,10757,117,primaria incompleta,primaria
472147,472147,11,4,10757,117,sup. univ. incompleta,superior universitaria
472200,472200,11,1,10769,115,sup. univ. incompleta,superior universitaria


In [56]:
# Ahora generamos el código el 'id' de cada observación: conglomerado + vivienda + hogar + codigo persona

edu_2018['personal_id'] = edu_2018['conglome_18'].astype(str) + edu_2018['vivienda_18'].astype(str) + edu_2018['hogar_18'].astype(str) + edu_2018['codperso_18'].astype(str)

# Generamos también la variable 'id_vivienda': conglomerado + vivienda + hogar + codigo persona
edu_2018['vivienda_id'] = edu_2018['conglome_18'].astype(str) + edu_2018['vivienda_18'].astype(str) + edu_2018['hogar_18'].astype(str)
edu_2018

,index,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18,personal_id,vivienda_id
16,16,11,3,5636,20,primaria incompleta,primaria,563620113,56362011
28,28,11,3,7178,139,sup. no univ. completa,superior no universitaria,7178139113,717813911
35,35,11,3,10322,109,primaria completa,primaria,10322109113,1032210911
38,38,11,4,8690,6,primaria incompleta,primaria,86906114,8690611
56,56,11,5,6571,194,primaria incompleta,primaria,6571194115,657119411
...,...,...,...,...,...,...,...,...,...
472141,472141,11,3,10757,117,sup. univ. completa,superior universitaria,10757117113,1075711711
472145,472145,11,7,10757,117,primaria incompleta,primaria,10757117117,1075711711
472147,472147,11,4,10757,117,sup. univ. incompleta,superior universitaria,10757117114,1075711711
472200,472200,11,1,10769,115,sup. univ. incompleta,superior universitaria,10769115111,1076911511


In [57]:
# Ahora generamos el segundo sub grupo para el año 2018: personas que tienen secundaria completa y son recien egresadas de ese nivel

# Seleccionamos observaciones cuyo último nivel de estudios aprobado fue secundaria
secundaria = ['secundaria completa']
edu_2018_comp = edu_2018[edu.p301a_18.isin(secundaria) == True]

# Seleccionamos observaciones que el año anterior a la encuesta asistieron a secundaria
secundaria1 = ['secundaria']
edu_2018_comp = edu_2018_comp[edu.p304a_18.isin(secundaria1) == True]
edu_2018_comp.shape

C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/3728476757.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2018_comp = edu_2018[edu.p301a_18.isin(secundaria) == True]
C:\Users\JOSUED~1\AppData\Local\Temp/ipykernel_9608/3728476757.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  edu_2018_comp = edu_2018_comp[edu.p304a_18.isin(secundaria1) == True]


(1816, 9)

#### Trabajamos con la base 2019

In [58]:
#Borramos las filas con valores perdidos

edu_2019 = edu_2019.dropna()
edu_2019

,index,hogar_19,codperso_19,conglome_19,vivienda_19,p301a_19,p304a_19
357081,357081,11.0,3.0,5001.0,50.0,sup. univ. completa,superior universitaria
357082,357082,11.0,2.0,5001.0,7.0,sup. univ. incompleta,superior universitaria
357084,357084,11.0,6.0,5001.0,17.0,primaria incompleta,primaria
357086,357086,11.0,4.0,5001.0,50.0,secundaria incompleta,secundaria
357087,357087,11.0,4.0,5001.0,7.0,primaria incompleta,primaria
...,...,...,...,...,...,...,...
473656,473656,11.0,6.0,11350.0,3.0,educación inicial,educación inicial
473658,473658,11.0,3.0,11350.0,97.0,educación inicial,educación inicial
473659,473659,11.0,3.0,11350.0,3.0,primaria incompleta,primaria
473660,473660,11.0,2.0,11350.0,78.0,primaria incompleta,primaria


In [59]:
# Quitamos el .0 de hogar, codperso, conglome y vivienda

edu_2019['codperso_19'] = edu_2019['codperso_19'].astype(str).replace('\.0', '', regex=True)
edu_2019['vivienda_19'] = edu_2019['vivienda_19'].astype(str).replace('\.0', '', regex=True)
edu_2019['conglome_19'] = edu_2019['conglome_19'].astype(str).replace('\.0', '', regex=True)
edu_2019['hogar_19'] = edu_2019['hogar_19'].astype(str).replace('\.0', '', regex=True)
edu_2019

,index,hogar_19,codperso_19,conglome_19,vivienda_19,p301a_19,p304a_19
357081,357081,11,3,5001,50,sup. univ. completa,superior universitaria
357082,357082,11,2,5001,7,sup. univ. incompleta,superior universitaria
357084,357084,11,6,5001,17,primaria incompleta,primaria
357086,357086,11,4,5001,50,secundaria incompleta,secundaria
357087,357087,11,4,5001,7,primaria incompleta,primaria
...,...,...,...,...,...,...,...
473656,473656,11,6,11350,3,educación inicial,educación inicial
473658,473658,11,3,11350,97,educación inicial,educación inicial
473659,473659,11,3,11350,3,primaria incompleta,primaria
473660,473660,11,2,11350,78,primaria incompleta,primaria


In [61]:
# Ahora generamos el código el 'id' de cada observación: conglomerado + vivienda + hogar + codigo persona

edu_2019['personal_id'] = edu_2019['conglome_19'].astype(str) + edu_2019['vivienda_19'].astype(str) + edu_2019['hogar_19'].astype(str) + edu_2019['codperso_19'].astype(str)

# Generamos también la variable 'id_vivienda': conglomerado + vivienda + hogar + codigo persona
edu_2019['vivienda_id'] = edu_2019['conglome_19'].astype(str) + edu_2019['vivienda_19'].astype(str) + edu_2019['hogar_19'].astype(str)
edu_2019

,index,hogar_19,codperso_19,conglome_19,vivienda_19,p301a_19,p304a_19,personal_id,vivienda_id
357081,357081,11,3,5001,50,sup. univ. completa,superior universitaria,500150113,50015011
357082,357082,11,2,5001,7,sup. univ. incompleta,superior universitaria,50017112,5001711
357084,357084,11,6,5001,17,primaria incompleta,primaria,500117116,50011711
357086,357086,11,4,5001,50,secundaria incompleta,secundaria,500150114,50015011
357087,357087,11,4,5001,7,primaria incompleta,primaria,50017114,5001711
...,...,...,...,...,...,...,...,...,...
473656,473656,11,6,11350,3,educación inicial,educación inicial,113503116,11350311
473658,473658,11,3,11350,97,educación inicial,educación inicial,1135097113,113509711
473659,473659,11,3,11350,3,primaria incompleta,primaria,113503113,11350311
473660,473660,11,2,11350,78,primaria incompleta,primaria,1135078112,113507811


### Ahora generamos los merge en periodos de años seguidos

Para el periodo 2015-2016: merge: 'edu_2015_comp' + 'edu_2016'

Para el periodo 2016-2017: merge: 'edu_2016_comp' + 'edu_2017'

Para el periodo 2017-2018: merge: 'edu_2017_comp' + 'edu_2018'

Para el periodo 2018-2019: merge: 'edu_2018_comp' + 'edu_2019'

#### Merge 2015 - 2016

In [62]:
edu_2015_2016 = pd.merge(edu_2015_comp,
                        edu_2016,
                        on = ['personal_id','vivienda_id'],
                        how = 'inner', indicator = True)
edu_2015_2016

,index_x,hogar_15,codperso_15,conglome_15,vivienda_15,p301a_15,p304a_15,personal_id,vivienda_id,index_y,hogar_16,codperso_16,conglome_16,vivienda_16,p301a_16,p304a_16,_merge
0,3767,11,3,7955,24,secundaria completa,secundaria,795524113,79552411,3767,11,3,7955,24,sup. no univ. incompleta,superior no universitaria,both
1,5992,11,3,9159,136,secundaria completa,secundaria,9159136113,915913611,150679,11,3,9159,136,primaria incompleta,primaria,both
2,6672,11,6,8668,4,secundaria completa,secundaria,86684116,8668411,15369,11,6,8668,4,primaria incompleta,primaria,both
3,7589,11,3,415,3,secundaria completa,secundaria,4153113,415311,7589,11,3,415,3,secundaria completa,secundaria,both
4,9780,11,5,5175,56,secundaria completa,secundaria,517556115,51755611,9780,11,5,5175,56,sup. univ. incompleta,superior universitaria,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,442821,11,7,9113,156,secundaria completa,secundaria,9113156117,911315611,442821,11,7,9113,156,secundaria completa,superior no universitaria,both
228,443477,11,4,9145,1,secundaria completa,secundaria,91451114,9145111,443477,11,4,9145,1,secundaria completa,superior universitaria,both
229,457377,11,3,9855,132,secundaria completa,secundaria,9855132113,985513211,457377,11,3,9855,132,sup. univ. incompleta,superior universitaria,both
230,463268,11,2,10126,102,secundaria completa,secundaria,10126102112,1012610211,463268,11,2,10126,102,sup. univ. incompleta,superior universitaria,both


#### Merge 2016 - 2017

In [63]:
edu_2016_2017 = pd.merge(edu_2016_comp,
                        edu_2017,
                        on = ['personal_id','vivienda_id'],
                        how = 'inner', indicator = True)
edu_2016_2017

,index_x,hogar_16,codperso_16,conglome_16,vivienda_16,p301a_16,p304a_16,personal_id,vivienda_id,index_y,hogar_17,codperso_17,conglome_17,vivienda_17,p301a_17,p304a_17,_merge
0,3275,11,3,8603,123,secundaria completa,secundaria,8603123113,860312311,3275,11,3,8603,123,sup. no univ. incompleta,superior no universitaria,both
1,3487,11,2,9264,29,secundaria completa,secundaria,926429112,92642911,3487,11,2,9264,29,sup. univ. incompleta,superior universitaria,both
2,3729,11,4,7403,56,secundaria completa,secundaria,740356114,74035611,3729,11,4,7403,56,secundaria completa,secundaria,both
3,6107,11,3,5243,7,secundaria completa,secundaria,52437113,5243711,362334,11,3,5243,7,educación inicial,educación inicial,both
4,6775,11,3,6410,26,secundaria completa,secundaria,641026113,64102611,99670,11,3,6410,26,secundaria incompleta,secundaria,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,463336,11,4,10130,77,secundaria completa,secundaria,1013077114,101307711,463336,11,4,10130,77,sup. no univ. incompleta,superior no universitaria,both
242,465353,11,3,10216,44,secundaria completa,secundaria,1021644113,102164411,465357,11,3,10216,44,primaria incompleta,primaria,both
243,465468,11,3,10221,34,secundaria completa,secundaria,1022134113,102213411,465473,11,3,10221,34,secundaria incompleta,secundaria,both
244,468008,11,4,10327,34,secundaria completa,secundaria,1032734114,103273411,468008,11,4,10327,34,secundaria completa,superior universitaria,both


#### Merge 2017 - 2018

In [64]:
edu_2017_2018 = pd.merge(edu_2017_comp,
                        edu_2018,
                        on = ['personal_id','vivienda_id'],
                        how = 'inner', indicator = True)
edu_2017_2018

,index_x,hogar_17,codperso_17,conglome_17,vivienda_17,p301a_17,p304a_17,personal_id,vivienda_id,index_y,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18,_merge
0,2526,11,3,9605,147,secundaria completa,secundaria,9605147113,960514711,452534,11,3,9605,147,secundaria incompleta,secundaria,both
1,3729,11,4,7403,56,secundaria completa,secundaria,740356114,74035611,3729,11,4,7403,56,secundaria completa,secundaria,both
2,5964,11,3,10043,75,secundaria completa,secundaria,1004375113,100437511,461407,11,3,10043,75,secundaria incompleta,secundaria,both
3,13265,11,5,6832,37,secundaria completa,secundaria,683237115,68323711,13265,11,5,6832,37,sup. no univ. incompleta,superior no universitaria,both
4,14601,11,3,5878,142,secundaria completa,secundaria,5878142113,587814211,14601,11,3,5878,142,sup. univ. incompleta,superior universitaria,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,466071,11,4,10254,70,secundaria completa,secundaria,1025470114,102547011,466072,11,4,10254,70,primaria completa,primaria,both
276,468917,11,3,10370,103,secundaria completa,secundaria,10370103113,1037010311,468917,11,3,10370,103,sup. univ. incompleta,superior universitaria,both
277,469126,11,2,10381,56,secundaria completa,secundaria,1038156112,103815611,469126,11,2,10381,56,sup. univ. incompleta,superior universitaria,both
278,470804,11,2,10591,144,secundaria completa,secundaria,10591144112,1059114411,470804,11,2,10591,144,secundaria completa,superior no universitaria,both


#### Merge 2018 - 2019

In [65]:
edu_2018_2019 = pd.merge(edu_2018_comp,
                        edu_2019,
                        on = ['personal_id','vivienda_id'],
                        how = 'inner', indicator = True)
edu_2018_2019

,index_x,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18,personal_id,vivienda_id,index_y,hogar_19,codperso_19,conglome_19,vivienda_19,p301a_19,p304a_19,_merge
0,13666,11,2,8234,22,secundaria completa,secundaria,823422112,82342211,424551,11,2,8234,22,primaria incompleta,primaria,both
1,29607,11,2,9598,91,secundaria completa,secundaria,959891112,95989111,452408,11,2,9598,91,primaria completa,primaria,both
2,43879,11,3,5903,20,secundaria completa,secundaria,590320113,59032011,375637,11,3,5903,20,educación inicial,educación inicial,both
3,60603,11,2,9575,99,secundaria completa,secundaria,957599112,95759911,452041,11,2,9575,99,primaria incompleta,primaria,both
4,66457,11,2,10713,81,secundaria completa,secundaria,1071381112,107138111,471885,11,2,10713,81,secundaria incompleta,secundaria,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,468005,11,5,10327,34,secundaria completa,secundaria,1032734115,103273411,468005,11,5,10327,34,sup. univ. incompleta,superior universitaria,both
255,468518,11,4,10351,116,secundaria completa,secundaria,10351116114,1035111611,468521,11,4,10351,116,secundaria incompleta,secundaria,both
256,468762,11,3,10363,27,secundaria completa,secundaria,1036327113,103632711,468762,11,3,10363,27,sup. univ. incompleta,superior universitaria,both
257,471584,11,2,10670,45,secundaria completa,secundaria,1067045112,106704511,471584,11,2,10670,45,sup. univ. incompleta,superior universitaria,both


#### Ahora definiremos la funcion para saber si los estudiantes accedieron a educación superior al año siguiente de su egreso de educación secundaria

Necesitamos saber si los jóvenes recién egresados de secundaria accedieron a educación superior o no un año después haber egresado de secundaria.

Por ello, hasta el momento se han agrupado cuatro sub bases que permiten comparación: edu_2015_2016, edu_2016_2017, edu_2017_2018 y edu_2018_2019

En cada una de estas sub bases, necesitamos saber si en el segundo año, la persona logró acceder a educación superior. Por ejemplo, en el caso de la sub base edu_2015_2016, necesitamos saber si para el año 2016 accedió a educación superior.

Esta información nos brinda la variable p301a_1x, que tiene los siguientes valores: 'sup. no univ. incompleta', 'primaria incompleta', 'secundaria completa', 'sup. univ. incompleta', 'primaria completa', 'secundaria incompleta', 'sup. no univ. completa', 'sup. univ. completa'.

Dado que casi todas las variables tienen como tipo de dato 'objeto', el cual es complicado convertir a string para comparar, codificaré cada uno de los valores de p301a_1x a un número, para luego poder definir una función que permita saber si los jóvenes accedieron a educación superior. La codificación es la siguiente:

'educacion inicial = 1'
'primaria incompleta = 2'
'primaria completa = 3'
'secundaria incompleta = 4'
'secundaria completa = 5'
'sup. no univ. incompleta = 6'
'sup. no univ. completa = 7'
'sup. univ. incompleta = 8'
'sup. univ. completa = 9'
'posgrado = 10'

Cabe resaltar que esta codificación se realizará solo para el segundo año de cada sub grupo de dos años. Así por ejemplo, para la sub base edu_2015_2016, se codificará solo la variable p301a_16, correspondiente al segundo año. Precisamente porque nos interesa saber si al siguiente año se accedió a educación superior. Por tanto, en este caso, la variable p301a_15, correspondiente al primer año no es de interés, porque todos sus valores son 'secundaria completa', dado que son recien egresados de educación secundaria.

Esta operación se repetirá para las 3 sub bases.

In [66]:
edu_2015_2016.p301a_16.unique()

array(['sup. no univ. incompleta', 'primaria incompleta',
       'secundaria completa', 'sup. univ. incompleta',
       'primaria completa', 'secundaria incompleta',
       'sup. no univ. completa', 'sup. univ. completa'], dtype=object)

In [67]:
edu_2016_2017.p301a_17.unique()

array(['sup. no univ. incompleta', 'sup. univ. incompleta',
       'secundaria completa', 'educación inicial',
       'secundaria incompleta', 'primaria completa',
       'primaria incompleta', 'sup. no univ. completa',
       'sup. univ. completa', 'postgrado'], dtype=object)

In [68]:
edu_2017_2018.p301a_18.unique()

array(['secundaria incompleta', 'secundaria completa',
       'sup. no univ. incompleta', 'sup. univ. incompleta',
       'primaria completa', 'primaria incompleta', 'educación inicial'],
      dtype=object)

In [69]:
edu_2018_2019.p301a_19.unique()

array(['primaria incompleta', 'primaria completa', 'educación inicial',
       'secundaria incompleta', 'sup. univ. incompleta',
       'secundaria completa', 'sup. no univ. incompleta',
       'sup. no univ. completa'], dtype=object)

In [70]:
# Definimos el diccionario con los valores a codificar, dado que para cada uno de los años los valores difieren en orden y cantidad

valores_2016 = {'primaria incompleta':2,'primaria completa':3,
                'secundaria incompleta':4,'secundaria completa':5,
                'sup. no univ. incompleta':6,'sup. no univ. completa':7,
                'sup. univ. incompleta':8,'sup. univ. completa':9}

valores_2017 = {'educación inicial':1,'primaria incompleta':2,
                'primaria completa':3,'secundaria incompleta':4,
                'secundaria completa':5,'sup. no univ. incompleta':6,
                'sup. no univ. completa':7,'sup. univ. incompleta':8,
                'sup. univ. completa':9,'postgrado':10}

valores_2018 = {'educación inicial':1,'primaria incompleta':2,
                'primaria completa':3,'secundaria incompleta':4,
                'secundaria completa':5,'sup. no univ. incompleta':6,
                'sup. univ. incompleta':8}

valores_2019 = {'educación inicial':1, 'primaria incompleta':2,
                'primaria completa':3, 'secundaria incompleta':4,
                'secundaria completa':5, 'sup. no univ. incompleta':6,
               'sup. no univ. completa':7, 'sup. univ. incompleta':8}

In [71]:
# Reemplazamos los valores objeto por valores numéricos codificados en el diccionario

edu_2015_2016.p301a_16 = [valores_2016[item] for item in edu_2015_2016.p301a_16]
edu_2016_2017.p301a_17 = [valores_2017[item] for item in edu_2016_2017.p301a_17]
edu_2017_2018.p301a_18 = [valores_2018[item] for item in edu_2017_2018.p301a_18]
edu_2018_2019.p301a_19 = [valores_2019[item] for item in edu_2018_2019.p301a_19]

In [72]:
edu_2015_2016.p301a_16.unique()

array([6, 2, 5, 8, 3, 4, 7, 9], dtype=int64)

In [73]:
edu_2016_2017.p301a_17.unique()

array([ 6,  8,  5,  1,  4,  3,  2,  7,  9, 10], dtype=int64)

In [74]:
edu_2017_2018.p301a_18.unique()

array([4, 5, 6, 8, 3, 2, 1], dtype=int64)

In [75]:
edu_2018_2019.p301a_19.unique()

array([2, 3, 1, 4, 8, 5, 6, 7], dtype=int64)

Queremos saber si los jóvenes accedieron a educación superior. Por lo tanto, nos interesa que al año siguiente de cada periodo tengan, o bien, superior universitaria incompleta, o bien, superior no universitaria incompleta. Es decir, los interesa los valores 6 y 8.

In [76]:
#Definimos la función para saber si accedieron o no
def clasif_acceso(x):
    '''
    Define si un joven recién egresado de secundaria accedió a educación superior un año después de su egreso.
    Input:
        x = p301a_1x Esta es una variable que indica el último nivel que aprobó
    Output:
        acceso = es 1, cuando la persona accedió o bien a educación superior universitaria, o bien a educación superior no universitaria
                 es 0, cuando la persona no logró acceder
   '''
    n = x
    if n == 6:
        acceso = 1
    elif n == 8:
        acceso = 1
    else:
        acceso = 0
    return acceso

In [77]:
#Ahora aplicamos la función para generar la variable acceso en cada sub base
edu_2015_2016['acceso'] = edu_2015_2016['p301a_16'].apply(lambda x: clasif_acceso(x))
edu_2016_2017['acceso'] = edu_2016_2017['p301a_17'].apply(lambda x: clasif_acceso(x))
edu_2017_2018['acceso'] = edu_2017_2018['p301a_18'].apply(lambda x: clasif_acceso(x))
edu_2018_2019['acceso'] = edu_2018_2019['p301a_19'].apply(lambda x: clasif_acceso(x))

In [79]:
edu_2018_2019

,index_x,hogar_18,codperso_18,conglome_18,vivienda_18,p301a_18,p304a_18,personal_id,vivienda_id,index_y,hogar_19,codperso_19,conglome_19,vivienda_19,p301a_19,p304a_19,_merge,acceso
0,13666,11,2,8234,22,secundaria completa,secundaria,823422112,82342211,424551,11,2,8234,22,2,primaria,both,0
1,29607,11,2,9598,91,secundaria completa,secundaria,959891112,95989111,452408,11,2,9598,91,3,primaria,both,0
2,43879,11,3,5903,20,secundaria completa,secundaria,590320113,59032011,375637,11,3,5903,20,1,educación inicial,both,0
3,60603,11,2,9575,99,secundaria completa,secundaria,957599112,95759911,452041,11,2,9575,99,2,primaria,both,0
4,66457,11,2,10713,81,secundaria completa,secundaria,1071381112,107138111,471885,11,2,10713,81,4,secundaria,both,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,468005,11,5,10327,34,secundaria completa,secundaria,1032734115,103273411,468005,11,5,10327,34,8,superior universitaria,both,1
255,468518,11,4,10351,116,secundaria completa,secundaria,10351116114,1035111611,468521,11,4,10351,116,4,secundaria,both,0
256,468762,11,3,10363,27,secundaria completa,secundaria,1036327113,103632711,468762,11,3,10363,27,8,superior universitaria,both,1
257,471584,11,2,10670,45,secundaria completa,secundaria,1067045112,106704511,471584,11,2,10670,45,8,superior universitaria,both,1


Ahora que ya tenemos la variable 'acceso', generaremos dataframes para los años 2015, 2016 y 2017, pero solo con las variables que nos interesan, entre ellas 'acceso'. Es decir, ahora sabremos si los jóvenes del año 2015 lograron acceder a educación superior en el año 2016, y así sucesivamente.

Las variables que nos interesa mantener son las siguientes:

'personal_id' = código de identificación personal,
'vivienda_id' = código de identificación de vivienda,
'acceso' = acceso a educación secundaria

In [80]:
#Creamos listas con los atributos de interés

l2015 = ['personal_id',
         'vivienda_id',
         'acceso']

l2016 = ['personal_id',
         'vivienda_id',
         'acceso']

l2017 = ['personal_id',
         'vivienda_id',
         'acceso']

l2018 = ['personal_id',
        'vivienda_id',
        'acceso']

In [81]:
#Generamos las bases por año, con los atributos de interés y 'acceso'

edu_2015_acceso = edu_2015_2016[l2015].copy().reset_index()
edu_2016_acceso = edu_2016_2017[l2016].copy().reset_index()
edu_2017_acceso = edu_2017_2018[l2017].copy().reset_index()
edu_2018_acceso = edu_2018_2019[l2018].copy().reset_index()

In [82]:
#Creamos la variable año para cada una de las bases

edu_2015_acceso['year'] = '2015'
edu_2016_acceso['year'] = '2016'
edu_2017_acceso['year'] = '2017'
edu_2018_acceso['year'] = '2018'

In [83]:
# Ahora unificamos el nombre de todas las variables

edu_2015_acceso.rename(columns = {'p207_15': 'sexo','p300a_15': 'lengua'}, inplace=True)
edu_2016_acceso.rename(columns = {'p207_16': 'sexo','p300a_16': 'lengua'}, inplace=True)
edu_2017_acceso.rename(columns = {'p207_17': 'sexo','p300a_17': 'lengua'}, inplace=True)
edu_2018_acceso.rename(columns = {'p207_17': 'sexo','p300a_17': 'lengua'}, inplace=True)

In [85]:
edu_2018_acceso

,index,personal_id,vivienda_id,acceso,year
0,0,823422112,82342211,0,2018
1,1,959891112,95989111,0,2018
2,2,590320113,59032011,0,2018
3,3,957599112,95759911,0,2018
4,4,1071381112,107138111,0,2018
...,...,...,...,...,...
254,254,1032734115,103273411,1,2018
255,255,10351116114,1035111611,0,2018
256,256,1036327113,103632711,1,2018
257,257,1067045112,106704511,1,2018


In [87]:
# Ahora hacemos append para juntar las sub bases de los 3 años en una sola base
data = pd.concat([edu_2015_acceso, edu_2016_acceso, edu_2017_acceso, edu_2018_acceso], ignore_index=True)

In [88]:
data

,index,personal_id,vivienda_id,acceso,year
0,0,795524113,79552411,1,2015
1,1,9159136113,915913611,0,2015
2,2,86684116,8668411,0,2015
3,3,4153113,415311,0,2015
4,4,517556115,51755611,1,2015
...,...,...,...,...,...
1012,254,1032734115,103273411,1,2018
1013,255,10351116114,1035111611,0,2018
1014,256,1036327113,103632711,1,2018
1015,257,1067045112,106704511,1,2018


In [89]:
# Exportamos a csv

data.to_csv(r'C:\Users\Josue Daniel\Documents\Python\Trabajo_ML\data.csv', index = False)

In [90]:
data = pd.read_csv('data.csv', encoding = "latin-1")

In [91]:
data

,index,personal_id,vivienda_id,acceso,year
0,0,795524113,79552411,1,2015
1,1,9159136113,915913611,0,2015
2,2,86684116,8668411,0,2015
3,3,4153113,415311,0,2015
4,4,517556115,51755611,1,2015
...,...,...,...,...,...
1012,254,1032734115,103273411,1,2018
1013,255,10351116114,1035111611,0,2018
1014,256,1036327113,103632711,1,2018
1015,257,1067045112,106704511,1,2018
